# Checkpoints
When training models that take significant amounts of time to train (and also for reusing an already trained model) checkpointing becomes an invaluable tool. Checkpoints are snapshots of the networks weights and all other global model variables that are saved in regular intervals. We can also use them to load the network when using it after training. 
The checkpointing utilities are part of the `tf.train` module. 

To get consistent counting of the optimization steps, we need to start doing the step counting inside the tensorflow model instead of simply using the loop counter of the training loop. This can also easily be achieved with the `tf.train` utilities.

We have added a `RESTORE` variable which determines whether tf should initially look for a checkpoint to continue training or start from scratch. To test this, you need to restart the notebooks kernel to reset the tensorflow session.

In [ ]:
import tensorflow as tf
import tensorflow.contrib.learn as tflearn
import numpy as np

NUM_STEPS = 10
RESTORE = False

## Model
The network code remains the same.

In [ ]:
x = tf.placeholder(tf.float32, (None, 784), name="x")
tf.summary.image("image", tf.reshape(x, (-1, 28, 28, 1)))

y = tf.placeholder(tf.int64, (None), name="y")

W = tf.Variable(np.random.random((784, 10)), dtype=tf.float32, name="W")
b = tf.Variable(np.random.random(10), dtype=tf.float32, name="b")

l = tf.matmul(x, W) + b
tf.summary.histogram("logits", l)

p = tf.nn.softmax(l)
tf.summary.histogram("probabilities", p)

In [ ]:
with tf.name_scope("loss_calculation"):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, depth=10), logits=l)
    loss = tf.reduce_mean(loss)
tf.summary.scalar("loss", loss)

# calculate accuracy
with tf.name_scope("accuracy_calculation"):
    is_correct = tf.equal(tf.argmax(l, axis=1), y)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float64))
tf.summary.scalar("accuracy", accuracy)



## Step Counting in Optimizer
In many models counting the training steps is quite easy. Each training step corresponds to exactly one execution of the training operation of the optimizer. As this is so ubiquitous, the `minimize` function provides a dedicated parameter to pass in a `global_step` variable which will be automatically be incremented by one when the SGD update is performed.

To create a global step variable there exists the `tf.train.create_global_step` function. It adds a new, non-trainable variable of integral type named `"global_step"` and registeres it as the step variable. There also exists a `get_or_create_global_step` function, which only creates the new step variable if it cannot find a pre-existing one.

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
global_step = tf.train.create_global_step()
train_op = optimizer.minimize(loss, global_step=global_step)

## Load the Dataset and Create a Session

In [ ]:
mnist = tflearn.datasets.load_dataset("mnist")
images = mnist.train.images
labels = mnist.train.labels

session = tf.InteractiveSession()
summaries = tf.summary.merge_all()
writer = tf.summary.FileWriter("checkpoint_demo", session.graph)

## Checkpoint Saver
To save a checkpoint we first need to create a `tf.train.Saver` object. A Saver is responsible for both saving and restoring (loading) of checkpoints. When saving a checkpoint, tensorflow creates several files containing model information, and an additional index file that contains a list of all checkpoints that tensorflow has saved in that directory. This is important, because it facilitates finding the latest checkpoint file.

In the code below, we check whether we want to restore or start training from scratch. For restoring, we first look up the filename of the latest checkpoint in the aforementioned index file and call `restore` for that. Since `restore` needs to set Variables, it needs access to the session in which we want to restore. As restore assigns values to all Variables, there is no need to run the initializer operation after restoring from a checkpoint.

In [ ]:
saver = tf.train.Saver()
if RESTORE:
    saver.restore(session, tf.train.latest_checkpoint("checkpoint_demo"))
else:
    tf.global_variables_initializer().run()

## Training Loop
The same as before. We currently only save a checkpoint after finishing this loop. A more involved approach would also save regularly from within the loop.

In [ ]:
for i in range(NUM_STEPS):
    summary, _, step = session.run([summaries, train_op, global_step], {x: images, y: labels})
    writer.add_summary(summary, step)

writer.close()

After the training loop we save the weights with the saver. Again, we need the session so the saver has access to the values of the variables. For saving we give a path to the save file without extension. As we also pass in the global step counter, which causes the filename to be appended with a step count suffix. Also notice that we pass in a folder an filename `checkpoint_demo/model` whereas to get the latest checkpoint we only pass in the folder `checkpoint_demo`. Restore actually needs the path to the actual file, so it will be something like `checkpoint_demo/model-10.ckpt`.

We also use the helper function `tf.train.get_global_step` that tries to read the Variable that defines the global step from the graph. The `tf.train.global_step` function then reads the actual value (as opposed to the tensor) of the global step.

In [ ]:
saver.save(session, "checkpoint_demo/model", tf.train.global_step(session, tf.train.get_global_step()))

## Closing Remarks
The strategy for loading a saved model we have employed here is typical for the task of checkpointing. We rebuild the same model in python and then only fill in the missing values for the weights. This only requires that the same weight exist in the model we build and in the model we load. In particular, we can change anything in the model that is not related to the weights and still load the old checkpoint. For example, you can add more summary operations to your code, and can still reuse old training checkpoints. 

Another strategy is to load both the computation graph and the variable values from files. This can be done using `tf.train.import_meta_graph` to create the Saver. This functions builds the graph as it was saved, and returns a Saver object that can be used to restore the weights. 
Note, however, that this strategy has the disadvantage that you do not have any python variables that hold the tensors you want to run or feed. You need to manually set the based on their names in the saved graph. 
For this reason, I would recommend to use the first strategy whenever possible, and only resort to load the graph if you do not have access to the python code creating the model.